In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
df = pd.read_csv("FuelConsumption(1).csv")

In [20]:
df.head()

,Year,MAKE,MODEL,VEHICLE CLASS,ENGINE SIZE,CYLINDERS,TRANSMISSION,FUEL,FUEL CONSUMPTION,COEMISSIONS
0,2000,ACURA,1.6EL,COMPACT,1.6,4,A4,X,10.5,216
1,2000,ACURA,1.6EL,COMPACT,1.6,4,M5,X,9.8,205
2,2000,ACURA,3.2TL,MID-SIZE,3.2,6,AS5,Z,13.7,265
3,2000,ACURA,3.5RL,MID-SIZE,3.5,6,A4,Z,15.0,301
4,2000,ACURA,INTEGRA,SUBCOMPACT,1.8,4,A4,X,11.4,230


In [21]:
df.info()
df.rename(columns = {'COEMISSIONS ' : 'COEMISSIONS'}, inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              639 non-null    int64  
 1   MAKE              639 non-null    object 
 2   MODEL             639 non-null    object 
 3   VEHICLE CLASS     639 non-null    object 
 4   ENGINE SIZE       639 non-null    float64
 5   CYLINDERS         639 non-null    int64  
 6   TRANSMISSION      639 non-null    object 
 7   FUEL              639 non-null    object 
 8   FUEL CONSUMPTION  639 non-null    float64
 9   COEMISSIONS       639 non-null    int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 50.0+ KB


In [22]:
from sklearn.preprocessing import OneHotEncoder

encoded_df = pd.get_dummies(df[['MAKE', 'MODEL', 'VEHICLE CLASS', 'TRANSMISSION', 'FUEL', 'COEMISSIONS']], columns=['MAKE', 'MODEL', 'VEHICLE CLASS', 'TRANSMISSION', 'FUEL', 'COEMISSIONS']).astype(int)
df.drop(columns=['MAKE', 'MODEL', 'VEHICLE CLASS', 'TRANSMISSION', 'FUEL', 'COEMISSIONS'], inplace=True)
df = pd.concat([df, encoded_df], axis=1)

In [23]:
df.head()

,Year,ENGINE SIZE,CYLINDERS,FUEL CONSUMPTION,MAKE_ACURA,MAKE_AUDI,MAKE_BMW,MAKE_BUICK,MAKE_CADILLAC,MAKE_CHEVROLET,...,COEMISSIONS_430,COEMISSIONS_435,COEMISSIONS_437,COEMISSIONS_444,COEMISSIONS_458,COEMISSIONS_472,COEMISSIONS_474,COEMISSIONS_495,COEMISSIONS_513,COEMISSIONS_582
0,2000,1.6,4,10.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2000,1.6,4,9.8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2000,3.2,6,13.7,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2000,3.5,6,15.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2000,1.8,4,11.4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

cond = ['ENGINE SIZE', 'CYLINDERS']
df.loc[:, cond] = scaler.fit_transform(df.loc[:, cond].astype(float))

y = df.loc[ :, 'FUEL CONSUMPTION']
x = df.drop(columns = ['FUEL CONSUMPTION', 'Year']).astype(float)

/tmp/ipykernel_201808/3753406368.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.11182045 -1.11182045  0.11946771  0.11946771 -1.11182045 -1.11182045
 -1.11182045  0.11946771  0.11946771 -1.11182045 -1.11182045  0.11946771
  0.11946771 -1.11182045 -1.11182045  0.11946771  0.11946771  0.11946771
  0.11946771  0.11946771  0.11946771  0.11946771  0.11946771  1.35075586
  1.35075586  0.11946771  0.11946771 -1.11182045 -1.11182045 -1.11182045
 -1.11182045  0.11946771  0.11946771  0.11946771  0.11946771  0.11946771
  0.11946771  0.11946771  0.11946771  0.11946771  0.11946771  0.11946771
  0.11946771  0.11946771  0.11946771  1.35075586  1.35075586  1.35075586
  1.35075586  1.35075586  3.81333218  0.11946771  0.11946771  1.35075586
  1.35075586  0.11946771  0.11946771  0.11946771  0.11946771  1.35075586
  0.11946771  0.11946771  0.11946771  0.11946771  0.11946771  0.11946771
  0.11946771  1.35075586  1.35075586  1.

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(x), np.array(y),random_state=42)

In [26]:
%%time
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV


k_val = np.arange(start = 1, stop = 100, step = 1)
neigh = KNeighborsRegressor()

regressor = GridSearchCV(estimator = neigh, param_grid = {'n_neighbors' : k_val}, cv = 10)
regressor.fit(X = X_train, y = y_train)

best_param = regressor.best_params_
best_score = regressor.best_score_

CPU times: user 8.02 s, sys: 257 ms, total: 8.27 s
Wall time: 3.7 s


In [27]:
print(f"The lowest MSE is using n_neighbor = {best_param['n_neighbors']}, and is : {best_score}")

The lowest MSE is using n_neighbor = 4, and is : 0.8782567335211745


In [28]:
from sklearn.metrics import mean_squared_error

neigh_ = KNeighborsRegressor(n_neighbors = best_param['n_neighbors'])

neigh_.fit(X_train, y_train)
y_pred = neigh_.predict(X_test)

print(f" The MSE for the test values is {mean_squared_error(y_test, y_pred)}")

 The MSE for the test values is 2.509847656249999


In [29]:
results = pd.DataFrame({'y_real': y_test.reshape(-1,), 'y_pred': y_pred.reshape(-1,)})
results

,y_real,y_pred
0,13.9,13.675
1,17.8,16.975
2,14.0,13.300
3,26.3,20.375
4,14.4,14.250
...,...,...
155,16.2,16.800
156,11.6,11.075
157,14.7,14.550
158,14.6,14.050


In [30]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [31]:
%%time
from sklearn.neural_network import MLPRegressor

param_grid = {
    'hidden_layer_sizes': np.arange(start = 1, stop = 50, step = 1)
}

mlp = MLPRegressor(max_iter = 500, early_stopping = True)

mlp_regressor = GridSearchCV(estimator = mlp, param_grid = param_grid, cv = 10)
mlp_regressor.fit(X = X_train, y = y_train.reshape(-1,))

best_mlp_score = mlp_regressor.best_score_
best_mlp_param = mlp_regressor.best_params_

CPU times: user 37min 59s, sys: 1h 2min 23s, total: 1h 40min 23s
Wall time: 14min 24s


In [32]:
print(f"The lowest MSE is using hidden_layer_sizes = {best_mlp_param['hidden_layer_sizes']}, and is : {best_mlp_score}")

The lowest MSE is using hidden_layer_sizes = 20, and is : 0.713733103596295


In [33]:
mlp_reg = MLPRegressor(hidden_layer_sizes = best_mlp_param['hidden_layer_sizes'], max_iter = 1000)

mlp_reg.fit(X_train, y_train.reshape(-1,))
y_pred_mlp = mlp_reg.predict(X_test)

print(f" The MSE for the test values is {mean_squared_error(y_test.reshape(-1,), y_pred_mlp)}")

 The MSE for the test values is 3.3661877706480112


In [34]:
results = pd.DataFrame({'y_real': y_test.reshape(-1,), 'y_pred': y_pred_mlp.reshape(-1,)})
results

,y_real,y_pred
0,13.9,12.647766
1,17.8,15.891517
2,14.0,12.963046
3,26.3,22.651253
4,14.4,11.449581
...,...,...
155,16.2,15.181795
156,11.6,10.721268
157,14.7,11.480291
158,14.6,13.991022
